In [1]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src') #This will enable reading the modules
from pathlib import Path
from fuzzy_sql.fuzzy_sql import *
from fuzzy_sql.long_query import LONG_QUERY


In [2]:
#set paths
root_dir=Path('/home/samer/projects/fuzzy_sql')
real_dir=os.path.join(root_dir,'data/longitudinal/ready/real')
meta_dir=os.path.join(root_dir,'data/longitudinal/ready/metadata')
syn_dir=os.path.join(root_dir,'data/longitudinal/ready/synthetic')


In [3]:
# Construct longitudinal query object based on the real data

rp_path=real_dir+"/b_sample.csv" #real parent (baseline) path 
rc_path=real_dir+"/l_sample.csv" #real child path
meta_path=meta_dir+"/sample.json" #metdata path

rp=load_csv(rp_path) 
rc=load_csv(rc_path) 
with open(meta_path) as f:
    meta=json.load(f) #metadata for the data 
rp=assign_dtype(rp, meta['parent'])
rc=assign_dtype(rc, meta['child'])


conn = sqlite3.connect('fuzzy_sql.db')
make_table('sample_r_b', rp, conn)
make_table('sample_r_l', rc, conn)


DFLT_PARAMS={
    'AGG_OPS':{'AVG':0.5, 'SUM':0.3, 'MAX':0.1, 'MIN':0.1 },
    'LOGIC_OPS':{'AND':0.9,'OR':0.1},
    'NOT_STATE':{'0':0.8, '1':0.2},
    'CAT_OPS':{'=':0.25, '<>':0.25, 'LIKE':0.15, 'IN':0.15, 'NOT LIKE':0.1, 'NOT IN':0.1},
    'CNT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0.1, '<=':0.1, '<>':0.1, 'BETWEEN':0.2, 'NOT BETWEEN':0.1},
    'DT_OPS':{'=':0.2, '>':0.1, '<':0.1, '>=':0, '<=':0, '<>':0.1, 'BETWEEN':0.2, 'IN':0.1, 'NOT BETWEEN':0.1, 'NOT IN':0.1},
    'LESS_GRP_VARS': False, # enforce bias in random queries toward smaller number of groupby vars. Default is no bias (i.e. uniform sampling)
    'LESS_CMP_VARS':False, # enforce bias in random queries toward small number of  comparison terms. Default is no bias (i.e. uniform sampling)
    'JOIN_CNDTN':{'WHERE':0.5, 'AND':0.5} #Use WHERE or AND with JOIN CLAUSE
}
self=LONG_QUERY(conn,'sample_r_b','sample_r_l', meta,DFLT_PARAMS)

Table sample_r_b already exists in the database
Table sample_r_l already exists in the database


In [4]:
# # Generate Single queries 
# rnd_query=self.make_single_agg_query()
# rnd_query=self.make_single_agg_query_w_aggfntn()
# rnd_query=self.make_single_fltr_query()
# rnd_query=self.make_single_aggfltr_query()
# rnd_query=self.make_single_aggfltr_query_w_aggfntn()

In [5]:

# Generate Random queries
many_syn=[] #this is a list of holding the random queries against each available version of the synthetic datasets
for i in range(1,2):
    sp_path=syn_dir+f"/b_sample_syn_0{i}.csv" #synthetic parent (baseline) path 
    sc_path=syn_dir+f"/l_sample_syn_0{i}.csv" #synthetic child (longitudinal) path 

    sp=load_csv(sp_path)  
    sc=load_csv(sc_path) 

    sp=assign_dtype(sp, meta['parent'])
    sc=assign_dtype(sc, meta['child'])

    # import real data into database
    make_table(f'sample_s_b_0{i}', sp, conn)
    make_table(f'sample_s_l_0{i}', sc, conn)


    # mltpl_queries=self.make_mltpl_twin_agg_query(3,f'sample_s_b_0{i}',f'sample_s_l_0{i}')
    # mltpl_queries=self.make_mltpl_twin_agg_query_w_aggfntn(3,f'sample_s_b_0{i}',f'sample_s_l_0{i}')
    # mltpl_queries=self.make_mltpl_twin_fltr_query(3,f'sample_s_b_0{i}',f'sample_s_l_0{i}')
    mltpl_queries=self.make_mltpl_twin_aggfltr_query(3,f'sample_s_b_0{i}',f'sample_s_l_0{i}') #this is a list of all unmatched random queries 
    # mltpl_queries=self.make_mltpl_twin_aggfltr_query_w_aggfntn(3,f'sample_s_b_0{i}',f'sample_s_l_0{i}')
    mltpl_scored_queries=self.calc_mltpl_dist_scores(mltpl_queries) #this is a list of all random matched and scored queries 
    many_syn.append(mltpl_scored_queries)


Table sample_s_b_01 already exists in the database
Table sample_s_l_01 already exists in the database
Generated Random Aggregate Filter Query - 0 
Generated Random Aggregate Filter Query - 1 
Generated Random Aggregate Filter Query - 2 




In [19]:
mltpl_scored_queries[0]['query_syn']

,Homeless,PL_UR_CA,DX1,NEOMAT,COUNT(*)
0,0,1,1002,0,1
1,0,1,1009,0,1
2,0,1,101,0,1
3,0,1,1049,0,1
4,0,1,1083,0,1
...,...,...,...,...,...
16691,0,3,V5789,0,5
16692,0,3,V5811,0,5
16693,0,3,V5842,0,1
16694,0,3,V5878,0,0


In [20]:
mltpl_scored_queries[0]['query_real']

,Homeless,PL_UR_CA,DX1,NEOMAT,COUNT(*)
0,0,1,1002,0,0
1,0,1,1009,0,1
2,0,1,101,0,2
3,0,1,1049,0,1
4,0,1,1083,0,1
...,...,...,...,...,...
16691,0,3,V5789,0,5
16692,0,3,V5811,0,9
16693,0,3,V5842,0,0
16694,0,3,V5878,0,1


In [22]:
len(many_syn)

1

In [8]:
""" def print_twin_agg_queries(queries: dict, file_writer):
    all_real = queries['query_real']
    all_syn = queries['query_syn']
    all_params = queries['query_params']
    all_hlngr = queries['hlngr_dist']
    all_ecldn = queries['ecldn_dist']
    for real, syn, params, hlngr, ecldn in zip(all_real, all_syn, all_params, all_hlngr, all_ecldn):
        n_real_records=params['real_n_rcrds']
        n_syn_records=params['syn_n_rcrds']
        # get any 5 records to display from both real and syn queries
        a = list(range(len(real)))
        b = random.sample(a, min(5, len(a)))
        real = real.iloc[b]
        syn = syn.iloc[b]
        real_style = real.style.set_table_attributes("style='display:inline; margin-right: 20px; font-size: 8pt; text-align:center'").set_caption("Real")
        syn_style = syn.style.set_table_attributes("style='display:inline; font-size: 8pt;text-align:center'").set_caption("Synthetic")

        if ecldn > 100:
            real_style.set_properties(**{'color': 'blue'})
            syn_style.set_properties(**{'color': 'blue'})

        if hlngr > 0.5:
            real_style.set_properties(**{'color': 'red'})
            syn_style.set_properties(**{'color': 'red'})

        file_writer.write(real_style.to_html())
        file_writer.write('<br><br>')
        file_writer.write(syn_style.to_html())

        #SMK CHECK
        # if not pd.isnull(params['query_agg_op']):
        file_writer.write('<p style="font-size: 8pt; margin-bottom:-10px"><u>SQL for Real:</u></p>')
        #expr = params['query_tmplt'].format(*params['query_cat_vars'], params['query_agg_op'], params['query_cnt_var'], params['real_table_name'], *params['query_cat_vars'])
        file_writer.write('<p style="font-size: 8pt">{}</p>'.format(params['real_sql']))
        
        file_writer.write('<p style="font-size: 8pt; margin-top:-5px"> Resulted in {} records</p>'.format(str(n_real_records)))
        
        file_writer.write('<p style="font-size: 8pt;margin-bottom:-10px"><u>SQL for Synthetic:</u></p>')
        #expr = params['query_tmplt'].format(*params['query_cat_vars'], params['query_agg_op'], params['query_cnt_var'], params['syn_table_name'], *params['query_cat_vars'])
        file_writer.write('<p style="font-size: 8pt">{}</p>'.format(params['syn_sql']))
        
        file_writer.write('<p style="font-size: 8pt; margin-top:-5px"> Resulted in {} records</p>'.format(str(n_syn_records)))
        
        file_writer.write('<p style="font-size: 8pt"> Normalized Euclidean distance for ({}): {} </p>'.format( params['query_cnt_var'], str(np.round(ecldn, 2))))
        # else:
        #     file_writer.write('<p style="font-size: 8pt; margin-bottom:-10px"><u>SQL for Real:</u></p>')
        #     expr = params['query_tmplt'].format(*params['query_cat_vars'], params['real_table_name'], *params['query_cat_vars'])
        #     file_writer.write('<p style="font-size: 8pt">{}</p>'.format(expr))
            
        #     file_writer.write('<p style="font-size: 8pt; margin-top:-5px"> Resulted in {} records</p>'.format(str(n_real_records)))
            
        #     file_writer.write('<p style="font-size: 8pt;margin-bottom:-10px"><u>SQL for Synthetic:</u></p>')
        #     expr = params['query_tmplt'].format(*params['query_cat_vars'], params['syn_table_name'], *params['query_cat_vars'])
        #     file_writer.write('<p style="font-size: 8pt">{}</p>'.format(expr))
            
        #     file_writer.write('<p style="font-size: 8pt; margin-top:-5px"> Resulted in {} records</p>'.format(str(n_syn_records)))

        file_writer.write('<p style="font-size: 8pt"> Hellinger Distance: {} </p>'.format(str(np.round(hlngr, 3))))
        file_writer.write("<p>===========================================</p>")

    file_writer.write('<br>')
    hlngr_stats=calc_stats(np.array(all_hlngr))
    file_writer.write('<p style="font-size: 10pt; page-break-before:always"> Hellinger Distance Summary: \n  {} </p>'.format(str(hlngr_stats)))
    if not pd.isnull(params['query_agg_op']):
        ecldn_stats=calc_stats(np.array(all_ecldn))
        file_writer.write('<p style="font-size: 10pt"> Euclidean distance Summary : \n {} </p>'.format(str(ecldn_stats)))
    file_writer.write("<p>===========================================</p>")
    return file_writer """